In [1]:
%%pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *

''' orderSchema = StructType([
    StructField("SalesOrderNumber", StringType()),
    StructField("SalesOrderLineNumber", IntegerType()),
    StructField("OrderDate", DateType()),
    StructField("CustomerName", StringType()),
    StructField("Email", StringType()),
    StructField("Item", StringType()),
    StructField("Quantity", IntegerType()),
    StructField("UnitPrice", FloatType()),
    StructField("Tax", FloatType())
    ]) '''
    
df = spark.read.load('abfss://files@datalakecpvl0xw.dfs.core.windows.net/sales_data/dbo.DimFactRetail.txt', 
format='CSV',header=True
)
display(df.limit(10))

StatementMeta(, , -1, SessionStarting, , SessionStarting)

In [ ]:
%%pyspark
from pyspark.sql.functions import * #year,month, col, split
RetailSales = spark.read.load("abfss://files@datalakecpvl0xw.dfs.core.windows.net/sales_data/dbo.DimFactRetail.txt",
format = 'CSV',header=True,inferSchema=True)

display(RetailSales.limit(5))

StatementMeta(, , , Waiting, , Waiting)

In [ ]:
from pyspark.sql import functions as sf
#from pyspark.sql import functions as sf
transform_df = RetailSales.withColumn('FullName',
                                     sf.concat_ws(' ',sf.col('FirstName'),sf.col('MiddleName'),sf.col('LastName')))
display(transform_df.limit(10));
transform_df = transform_df.drop('FirstName','MiddleName','LastName')
transform_df.write.mode('overwrite').parquet('/retaildata/destination/TransformedRetailData/FactRetail.parquet')
print ("Transformed data saved!")


StatementMeta(, , , Waiting, , Waiting)

In [ ]:
from pyspark.sql.functions import *

partitioned_df = transform_df.withColumn('Year',year(col('OrderDate'))).withColumn('Month', month(col('OrderDate')))
display(partitioned_df.limit(10))
partitionedDate_df = partitioned_df.write.partitionBy('Year').mode('overwrite').parquet('/retaildata/destination/PartitionedRetailData')
print("partioned data saved")


StatementMeta(, , , Waiting, , Waiting)

In [ ]:
FactRetail2010 = spark.read.load('/retaildata/destination/PartitionedRetailData/Year=2010')
OderedFactRetail2010 = FactRetail2010.orderBy(FactRetail2010.SalesAmount.desc())
display(OderedFactRetail2010.limit(10))

StatementMeta(, , , Waiting, , Waiting)